# APA - Basic Usage Example

This notebook demonstrates the basic usage of the APA (Advanced Pavement Analytics) pipeline.

## Overview

APA is a geospatial AI pipeline that uses satellite imagery to predict the Pavement Condition Index (PCI) of urban roads.

## Prerequisites

Make sure you have:
1. Installed APA and its dependencies
2. A valid configuration file
3. Access to hyperspectral imagery data
4. Ground truth PCI data


In [2]:
# Import required libraries
import sys
from pathlib import Path

# Robust path configuration - works from any directory
def setup_apa_paths():
    """Setup APA paths to work from any directory."""
    # Find the project root by looking for src/ directory
    current_path = Path.cwd()
    
    # Try different possible locations
    possible_roots = [
        current_path,  # If running from project root
        current_path.parent,  # If running from examples/
        current_path.parent.parent,  # If running from examples/subdir
    ]
    
    for root in possible_roots:
        src_path = root / "src"
        if src_path.exists() and (src_path / "apa").exists():
            return root, src_path
    
    # Fallback: use current directory
    return current_path, current_path / "src"

# Setup paths
project_root, src_path = setup_apa_paths()
sys.path.insert(0, str(src_path))

print(f"🔧 Working directory: {Path.cwd()}")
print(f"📁 Project root: {project_root}")
print(f"📂 Source path: {src_path}")
print(f"✅ Source path exists: {src_path.exists()}")
print(f"✅ APA package exists: {(src_path / 'apa').exists()}")

# Import APA modules
import apa
from apa.config.manager import ConfigManager
from apa.pipeline.runner import APAPipeline
from apa.utils.visualization import VisualizationUtils
from apa.utils.metrics import MetricsCalculator


🔧 Working directory: /home/ariep/Hyperspectral Road/brach_from_github/apa/examples
📁 Project root: /home/ariep/Hyperspectral Road/brach_from_github/apa
📂 Source path: /home/ariep/Hyperspectral Road/brach_from_github/apa/src
✅ Source path exists: True
✅ APA package exists: True


In [5]:

# --- Enhanced Imports & Setup ---
import numpy as np
import logging
from pathlib import Path

# Ensure src is in path (in case this cell runs independently)
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import common API components
from apa.common import DataContainer, ProcessingResult, ModelResult, PipelineResult

# Import modules (now with lazy loading - no execution on import!)
from apa.modules import (
    HyperspectralDataImporter,
    GroundTruthDataImporter,
    ROIProcessor,
    RoadExtractor,
    PCISegmenter,
    DataPreprocessor,
    UNetModel,
    CNNModel,
    ModularPipeline
)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("apa_demo")

print("✅ Imports completed successfully")
print("📦 All modules are now lazy-loaded and ready to use")



✅ Imports completed successfully
📦 All modules are now lazy-loaded and ready to use


In [6]:
# --- Module Instance Creation ---
print("🔧 Creating module instances...")

# 1. Data Importers
hyperspectral_importer = HyperspectralDataImporter({
    'input_path': 'data/hyperspectral',
    'filename_NED': 'NED.h5',
    'filename_RGB': 'RGB.h5'
})

ground_truth_importer = GroundTruthDataImporter({
    'excel_path': 'data/Detroit/Pavement_Condition.csv'
})

# 2. Data Processors
roi_processor = ROIProcessor({
    'roi_bounds': [42.34429, 42.3917, -83.14294, -83.00007]  # Detroit ROI
})

road_extractor = RoadExtractor()
pci_segmenter = PCISegmenter()
data_preprocessor = DataPreprocessor({
    'crop_size': 64,
    'overlap': 0.2
})

# 3. Machine Learning Models
# unet_model = UNetModel({
#     'input_size': (64, 64, 12),
#     'n_classes': 4,
#     'epochs': 10,
#     'batch_size': 32,
#     'learning_rate': 1e-3
# })

# cnn_model = CNNModel({
#     'input_size': (64, 64, 12),
#     'n_classes': 4,
#     'epochs': 10,
#     'batch_size': 32,
#     'learning_rate': 1e-3
# })

# 4. Pipeline
pipeline = ModularPipeline("detroit_pipeline")

print("✅ All module instances created successfully!")
print(f"📊 Created {len([hyperspectral_importer, ground_truth_importer, roi_processor, road_extractor, pci_segmenter, data_preprocessor, pipeline])} module instances")


🔧 Creating module instances...
✅ All module instances created successfully!
📊 Created 7 module instances


In [7]:
# --- Data Flow Demonstration ---
print("🌊 Demonstrating Data Flow Through APA Pipeline")
print("=" * 60)

# Create dummy hyperspectral data for demonstration
print("1️⃣ Creating dummy hyperspectral data...")
dummy_hyperspectral = np.random.rand(100, 100, 12).astype(np.float32)
dummy_lon = np.random.rand(100, 100) * 0.1 - 83.0
dummy_lat = np.random.rand(100, 100) * 0.1 + 42.3

# Create DataContainer (standardized data structure)
hyperspectral_data = DataContainer(
    data={
        'hyperspectral_image': dummy_hyperspectral,
        'longitude_matrix': dummy_lon,
        'latitude_matrix': dummy_lat
    },
    data_type='hyperspectral',
    metadata={
        'source': 'dummy_data',
        'roi_bounds': [42.34429, 42.3917, -83.14294, -83.00007]
    }
)

print(f"✅ Hyperspectral data created: {hyperspectral_data.get_shape()}")
print(f"📊 Data type: {hyperspectral_data.data_type}")
print(f"📍 Coordinate bounds: {hyperspectral_data.metadata.get('roi_bounds', 'N/A')}")

# Demonstrate data flow through processing stages
print("\n2️⃣ Processing data through pipeline stages...")

# Stage 1: ROI Processing
print("   🔄 Stage 1: ROI Processing")
roi_result = roi_processor.process_data(hyperspectral_data, {
    'roi_bounds': [42.34429, 42.3917, -83.14294, -83.00007]
})

if roi_result.success:
    print(f"   ✅ ROI processing completed in {roi_result.processing_time:.2f}s")
    print(f"   📏 Output shape: {roi_result.processed_data.get_shape()}")
    current_data = roi_result.processed_data
else:
    print(f"   ❌ ROI processing failed: {roi_result.error_message}")
    current_data = hyperspectral_data

# Stage 2: Road Extraction
print("   🔄 Stage 2: Road Extraction")
road_result = road_extractor.process_data(current_data, {})

if road_result.success:
    print(f"   ✅ Road extraction completed in {road_result.processing_time:.2f}s")
    print(f"   🛣️ Road coverage: {road_result.processed_data.metadata.get('road_coverage', 'N/A')}")
    current_data = road_result.processed_data
else:
    print(f"   ❌ Road extraction failed: {road_result.error_message}")

# Stage 3: PCI Segmentation
print("   🔄 Stage 3: PCI Segmentation")
pci_result = pci_segmenter.process_data(current_data, {})

if pci_result.success:
    print(f"   ✅ PCI segmentation completed in {pci_result.processing_time:.2f}s")
    print(f"   📊 PCI distribution: {pci_result.processed_data.metadata.get('pci_distribution', 'N/A')}")
    current_data = pci_result.processed_data
else:
    print(f"   ❌ PCI segmentation failed: {pci_result.error_message}")

# Stage 4: Data Preprocessing
# print("   🔄 Stage 4: Data Preprocessing")
# preprocessing_result = data_preprocessor.process_data(current_data, {
#     'crop_size': 64,
#     'overlap': 0.2
# })

# if preprocessing_result.success:
#     print(f"   ✅ Data preprocessing completed in {preprocessing_result.processing_time:.2f}s")
#     print(f"   🔢 Number of segments: {preprocessing_result.processed_data.metadata.get('num_segments', 'N/A')}")
#     current_data = preprocessing_result.processed_data
# else:
#     print(f"   ❌ Data preprocessing failed: {preprocessing_result.error_message}")

print("\n✅ Data flow demonstration completed!")
print("📈 Data successfully transformed through all processing stages")


2025-10-25 16:57:04,988 - apa.roi_processor - ERROR - Error in ROI processing: APA Error: No data found within ROI bounds
2025-10-25 16:57:04,988 - apa.roi_processor - ERROR - Error in ROI processing: APA Error: No data found within ROI bounds
2025-10-25 16:57:04,989 - apa.roi_processor - ERROR - Error processing data: APA Error: ROI processing failed: APA Error: No data found within ROI bounds (Processing stage: roi_processing)
2025-10-25 16:57:04,989 - apa.roi_processor - ERROR - Error processing data: APA Error: ROI processing failed: APA Error: No data found within ROI bounds (Processing stage: roi_processing)
2025-10-25 16:57:04,989 - apa.road_extractor - ERROR - Error in road extraction: name 'np' is not defined
2025-10-25 16:57:04,989 - apa.road_extractor - ERROR - Error in road extraction: name 'np' is not defined
2025-10-25 16:57:04,990 - apa.road_extractor - ERROR - Error processing data: APA Error: Road extraction failed: name 'np' is not defined (Processing stage: road_extr

🌊 Demonstrating Data Flow Through APA Pipeline
1️⃣ Creating dummy hyperspectral data...
✅ Hyperspectral data created: (3,)
📊 Data type: hyperspectral
📍 Coordinate bounds: [42.34429, 42.3917, -83.14294, -83.00007]

2️⃣ Processing data through pipeline stages...
   🔄 Stage 1: ROI Processing
   ❌ ROI processing failed: APA Error: ROI processing failed: APA Error: No data found within ROI bounds (Processing stage: roi_processing)
   🔄 Stage 2: Road Extraction
   ❌ Road extraction failed: APA Error: Road extraction failed: name 'np' is not defined (Processing stage: road_extraction)
   🔄 Stage 3: PCI Segmentation
   ❌ PCI segmentation failed: APA Error: Data must be road_extracted or processed_ground_truth type

✅ Data flow demonstration completed!
📈 Data successfully transformed through all processing stages


In [ ]:
# --- Pipeline Orchestration ---
print("🚀 Setting up Modular Pipeline")
print("=" * 50)

# Add stages to the pipeline with dependencies
pipeline.add_custom_stage('roi_processing', roi_processor)
pipeline.add_custom_stage('road_extraction', road_extractor, dependencies=['roi_processing'])
pipeline.add_custom_stage('pci_segmentation', pci_segmenter, dependencies=['road_extraction'])
pipeline.add_custom_stage('data_preparation', data_preprocessor, dependencies=['pci_segmentation'])

# Get pipeline information
pipeline_info = pipeline.get_pipeline_info()
print(f"📋 Pipeline: {pipeline_info['pipeline_name']}")
print(f"🔧 Stages: {pipeline_info['stages']}")
print(f"📊 Execution order: {pipeline_info['execution_order']}")
print(f"🔗 Dependencies: {pipeline_info['dependencies']}")

# Run the complete pipeline
print("\n🌊 Running Complete Pipeline...")
pipeline_config = {
    'roi_processing': {
        'roi_bounds': [42.34429, 42.3917, -83.14294, -83.00007]
    },
    'road_extraction': {},
    'pci_segmentation': {},
    'data_preparation': {
        'crop_size': 64,
        'overlap': 0.2
    }
}

# Execute the pipeline
pipeline_result = pipeline.run_pipeline(hyperspectral_data, pipeline_config)

if pipeline_result.success:
    print(f"✅ Pipeline completed successfully in {pipeline_result.total_time:.2f}s")
    print(f"📊 Stages completed: {len(pipeline_result.stage_results)}")
    
    # Show results for each stage
    for stage_name, stage_result in pipeline_result.stage_results.items():
        status = "✅ SUCCESS" if stage_result.success else "❌ FAILED"
        time_taken = getattr(stage_result, 'processing_time', 'N/A')
        print(f"   {stage_name}: {status} ({time_taken}s)")
else:
    print(f"❌ Pipeline failed: {pipeline_result.error_message}")

print("\n🎉 Pipeline orchestration demonstration completed!")


# 🌊 APA Data Flow Architecture

## Data Flow Explanation

The APA pipeline follows a standardized data flow pattern where data moves through different stages using the `DataContainer` class. Here's how data flows from one module to the next:

### 1. **DataContainer Structure**
```python
DataContainer(
    data={
        'hyperspectral_image': np.ndarray,    # Main data
        'longitude_matrix': np.ndarray,       # Coordinate data
        'latitude_matrix': np.ndarray        # Coordinate data
    },
    data_type='hyperspectral',               # Data type identifier
    metadata={...},                          # Processing metadata
    source='data/source.h5'                  # Data source
)
```

### 2. **Processing Flow**
```
Raw Data → DataContainer → Processing → ProcessingResult → Next Stage
```

### 3. **Stage-by-Stage Data Transformation**

#### **Stage 1: Data Import**
- **Input**: File paths, configuration
- **Output**: `DataContainer` with hyperspectral imagery
- **Data Type**: `'hyperspectral'`
- **Key Data**: `hyperspectral_image`, coordinate matrices

#### **Stage 2: ROI Processing**
- **Input**: `DataContainer` (hyperspectral)
- **Output**: `ProcessingResult` → `DataContainer` (processed_hyperspectral)
- **Data Type**: `'processed_hyperspectral'`
- **Transformation**: Crops data to region of interest
- **Key Data**: Cropped hyperspectral image, updated coordinates

#### **Stage 3: Road Extraction**
- **Input**: `DataContainer` (processed_hyperspectral)
- **Output**: `ProcessingResult` → `DataContainer` (road_extracted)
- **Data Type**: `'road_extracted'`
- **Transformation**: Adds road mask to data
- **Key Data**: Original data + `road_mask`, `road_coverage` metadata

#### **Stage 4: PCI Segmentation**
- **Input**: `DataContainer` (road_extracted)
- **Output**: `ProcessingResult` → `DataContainer` (pci_segmented)
- **Data Type**: `'pci_segmented'`
- **Transformation**: Adds PCI segmentation map
- **Key Data**: Original data + `pci_segmentation`, `pci_distribution` metadata

#### **Stage 5: Data Preprocessing**
- **Input**: `DataContainer` (pci_segmented)
- **Output**: `ProcessingResult` → `DataContainer` (preprocessed)
- **Data Type**: `'preprocessed'`
- **Transformation**: Creates cropped segments for neural network
- **Key Data**: `cropped_segments` array, `num_segments` metadata

### 4. **Key Benefits of This Architecture**

✅ **Modularity**: Each stage is independent and can be used separately  
✅ **Composability**: Stages can be easily combined into custom pipelines  
✅ **Consistency**: All stages use the same data structures  
✅ **Validation**: Built-in input/output validation  
✅ **Error Handling**: Comprehensive error reporting  
✅ **Metadata**: Rich metadata tracking throughout the pipeline  

### 5. **Data Flow Visualization**
```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   Raw Data      │───▶│  DataContainer  │───▶│  Processing     │
│   (Files)       │    │  (Standardized) │    │  (Module)       │
└─────────────────┘    └─────────────────┘    └─────────────────┘
                                                         │
                                                         ▼
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  Next Stage     │◀───│ ProcessingResult│◀───│  Processed    │
│   (Module)       │    │  (Results)      │    │  DataContainer  │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

This architecture ensures that data flows smoothly through the pipeline while maintaining type safety, validation, and comprehensive error handling at each stage.
